In [54]:
import nltk
import string
import itertools
import os
import pickle
import itertools
import numpy as np

In [2]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/march/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/march/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Métricas

In [3]:
def freq_one(word, tokens):
    'Frecuencia de una palabra en un texto (Church 1990)'
    return sum((1 for t in tokens if t == word)) / float(len(tokens))

In [4]:
def freq_two(first, second, tokens, win_size=5):
    '''
    Frecuencia de dos palabras en un texto (Church 1990)
    
    Aplica correccion de (w - 1) para compensar por múltiples conteos.
    
    O(|tokens| * win_size)
    '''
    count = 0
    for idx in range(len(tokens)):
        left = max(0, idx - win_size + 1)
        if tokens[idx] == second:
            for x in tokens[left:idx]:
                if x == first:
                    count += 1
    return count / (win_size - 1) / float(len(tokens))

In [5]:
def freq_two_fast(first_token_pos, second_token_pos, token_count, win_size=5):
    '''
    Equivalente a freq_two pero más veloz.
    
    Args:
        first_token_pos :: [int] lista de posiciones en el texto donde se encuentra el token a izq
        second_token_pos :: [int] lista de posiciones en el texto donde se encuentra el token a der
        token_count :: int cantidad te tokens en el texto
        
    O(#apariciones token 1 + #apariciones token 2)
    '''
    s_it = reversed(second_token_pos)
    f_it = reversed(first_token_pos)
    
    count = 0
    
    try:
        f_idx = next(f_it)
        s_idx = next(s_it)
        while True:
            while (f_idx > s_idx):
                f_idx = next(f_it)
            local_f_it, f_it = itertools.tee(f_it)
            local_f_idx = f_idx
            while (s_idx - local_f_idx < win_size):
                if (s_idx - local_f_idx > 0):
                    count += 1
                local_f_idx = next(local_f_it)
            s_idx = next(s_it)
    except StopIteration:
        return count / (win_size - 1) / token_count

In [6]:
def info(first, second, tokens, win_size=5):
    'Mutual information of two tokens (Church 1990)'
    f2 = freq_two(first, second, tokens, win_size)
    if f2 < 6:
        return None
    fa = freq_one(first, tokens)
    fb = freq_one(second, tokens)
    q = f2 / (fa * fb)
    return np.log2(q)

In [80]:
def q_fast(first_token_pos, second_token_pos, token_count, win_size=5):
    f2 = freq_two_fast(first_token_pos, second_token_pos, token_count, win_size)
    if f2 < (6 / (win_size - 1) / token_count):
        return None
    fa = len(first_token_pos)
    fb = len(second_token_pos)
    q = f2 / (fa * fb)
    return q 

### Corpus

In [8]:
punc_translator = str.maketrans(dict(zip(string.punctuation, [None] * len(string.punctuation))))
def tokenize(string):
    'Returns list of tokens'
    # TODO: Cambiar split por: https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
    words = string.split(' ')  # [Palabra]
    retoken = [ t.lower()
        for w in words
        for t in nltk.word_tokenize(w)
    ] 
    no_punt = [ t
        for t in retoken
        if t.translate(punc_translator) != ''
    ]
    return no_punt 

In [9]:
docno = []
corpus = []
toggle = 0
with open('ap/ap.txt','r') as f:
    for line in f:
        if line.startswith('<DOCNO>'):
            line_new = line.replace('<DOCNO> ','')
            line_new = line_new.replace(' </DOCNO>\n','')
            docno.append(line_new)
        if toggle:
            corpus.append(line)
            toggle = 0
        if line.startswith('<TEXT>'):
            toggle = 1        
docs = dict(zip(docno, corpus))      

#### Tests métricas

In [107]:
a = next(iter(docs.values()))

In [40]:
a_tokens = tokenize(a)

In [57]:
a_commons = [x[0] for x in Counter(sorted(a_tokens)).most_common(10)]

In [145]:
a_commons_pos = {
    token : [idx for idx, t in enumerate(a_tokens) if t == token]
    for token in a_commons
}

In [190]:
first = a_commons[0]
for j in range(10):
    second = a_commons[j]
    f = freq_two(first, second, a_tokens)
    f2 = freq_two_fast(a_commons_pos[first], a_commons_pos[second], len(a_tokens))
    print(first, second, f2 * len(a_tokens) * 4, f * len(a_tokens) * 4, f == f2)

the the 8.0 8.0 True
the a 3.0 3.0 True
the and 3.0 3.0 True
the said 4.0 4.0 True
the was 5.000000000000001 5.000000000000001 True
the of 7.0 7.0 True
the 's 7.0 7.0 True
the boy 9.0 9.0 True
the he 0.0 0.0 True
the school 6.0 6.0 True


###  Top 500

In [11]:
from collections import Counter

In [16]:
PICKLE_DIR = 'pickles'
EJ1_TOKENS_PATH = os.path.join(PICKLE_DIR, 'ej1_tokens.pkl')

In [17]:
if not os.path.isdir(PICKLE_DIR):
    os.mkdir(PICKLE_DIR)

In [26]:
if not os.path.isfile(EJ1_TOKENS_PATH):
    token_docs = {
        docno : tokenize(text)
        for docno, text in docs.items()
    }
    with open(EJ1_TOKENS_PATH, 'wb') as f:
        pickle.dump(token_docs, f)
else:
    with open(EJ1_TOKENS_PATH, 'rb') as f:
        token_docs = pickle.load(f)

In [27]:
all_tokens = [t for tokens in token_docs.values() for t in tokens]

In [30]:
top_tokens_counts = Counter(sorted(all_tokens)).most_common(500)

In [31]:
top_tokens = [x[0] for x in top_tokens_counts]

In [33]:
token_docs = {
    docno : tokens
    for docno, tokens in token_docs.items()
    if len(tokens) > 0
}

In [35]:
common_tokens_doc_pos = {
    docno : {
        token : [idx for idx, t in enumerate(text) if t == token]
        for token in top_tokens
    }
    for docno, text in token_docs.items()
}

In [37]:
docs_len = {
    docno : len(tokens)
    for docno, tokens in token_docs.items()
}

In [81]:
mutual_q = []

for i, first in enumerate(top_tokens):
    print(i)
    for j, second in enumerate(top_tokens):
        mutual_q.append(
            ((first, second), [q_fast(token_pos[first], token_pos[second], token_len) 
                               for token_pos, token_len in zip(common_tokens_doc_pos.values(), docs_len.values())])
        )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [102]:
for x in mutual_q:
    if any([y >= 1 for y in x[1] if y]):
        print(x)

    Consultar: P(x, y) / P(x) * P(y) parece ser < 1 siempre
    Respuesta: Hay que calcular f(x, y) para todo el corpus en su conjunto. 
    
    Consulta: ¿N pasa a ser sum(|d_i|)? (O sea, P(x, y) = FM(x, y) / N)

In [104]:
relations = {
    (first, second) : [sum(np.log2([i for i in infos if i is not None])) / len(token_docs)]
    for (first, second), infos in mutual_q
}

In [105]:
top_relations = sorted([r for r in relations.items() if r[1][0] != 0.0], 
                       key=lambda kv: kv[1], reverse=True)

In [106]:
top_relations[:15]

[(('american', 'health'), [-0.0052404015099068533]),
 (('me', 'down'), [-0.0056558031662215852]),
 (('or', 'percent'), [-0.0056768466336496789]),
 (('this', 'year'), [-0.0057524360471026652]),
 (('as', 'country'), [-0.0057967134819562754]),
 (('as', 'this'), [-0.0057967134819562754]),
 (('first', 'city'), [-0.0058346725806216497]),
 (('year', 'year'), [-0.0058709212624609422]),
 (('down', 'at'), [-0.0059771848497051664]),
 (('up', 'to'), [-0.0060300479411241389]),
 (('high', 'school'), [-0.0060650382650093444]),
 (('june', '1'), [-0.0060879114055722222]),
 (('in', '1987'), [-0.006117607049178333]),
 (('it', 'said'), [-0.0061662513923437193]),
 (('was', 'up'), [-0.0061943116119242796])]